In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 48.7 MB/s eta 0:00:00


In [2]:
import mysql.connector
from mysql.connector import Error

hostname = "j1gozd.h.filess.io"
database = "healthdbrdbms_periodpale"
port = "3307"
username = "healthdbrdbms_periodpale"
password = "91dd9ee319efa4d84881f8ec2df6f230d6486364"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



/tmp/ipython-input-709520993.py:13: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('healthdbrdbms_periodpale',)
MySQL connection is closed


In [5]:
import pandas as pd

try:
    drugproductsunfinished = pd.read_csv('/content/Drugs_unfinished_products.csv', encoding='latin-1')
except UnicodeDecodeError:
    drugproductsunfinished = pd.read_csv('/content/Drugs_unfinished_products.csv', encoding='ISO-8859-1')

In [9]:
drugproductsunfinished.dtypes

,0
PRODUCTID,object
PRODUCTNDC,object
PRODUCTTYPENAME,object
NONPROPRIETARYNAME,object
DOSAGEFORMNAME,object
STARTMARKETINGDATE,float64
ENDMARKETINGDATE,float64
MARKETINGCATEGORYNAME,object
LABELERNAME,object
SUBSTANCENAME,object


In [26]:
drugproductsunfinished.shape

(19042, 13)

In [25]:
import mysql.connector
from mysql.connector import Error

hostname = "j1gozd.h.filess.io"
database = "healthdbrdbms_periodpale"
port = "3307"
username = "healthdbrdbms_periodpale"
password = "91dd9ee319efa4d84881f8ec2df6f230d6486364"

csv = "/content/Drugs_unfinished_products.csv"
table = "Drugs_unfinished_products"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute(f"DROP TABLE IF EXISTS {table};")
        print("Table Dropped")
        print("Creating New Table")
        new_table =  f"""
        CREATE TABLE {table} (
            PRODUCTID varchar(200),
            PRODUCTNDC varchar(200),
            PRODUCTTYPENAME varchar(500),
            NONPROPRIETARYNAME TEXT,
            DOSAGEFORMNAME varchar(500),
            STARTMARKETINGDATE DOUBLE,
            ENDMARKETINGDATE	DOUBLE,
            MARKETINGCATEGORYNAME	VARCHAR(500),
            LABELERNAME	varchar(500),
            SUBSTANCENAME	TEXT,
            ACTIVE_NUMERATOR_STRENGTH	TEXT,
            ACTIVE_INGRED_UNIT	TEXT,
            DEASCHEDULE varchar(500)
            );
            """
        cursor.execute(new_table)
        print("Table Created")
        print("Inserting Data into df")
        try:
          data = pd.read_csv('/content/Drugs_unfinished_products.csv', encoding='latin-1')
        except UnicodeDecodeError:
          data = pd.read_csv('/content/Drugs_unfinished_products.csv', encoding='ISO-8859-1')
        print("Data Loaded into df")

        batch = 500
        records = len(data)

        print("Starting insertion")
        for i in range(0, records, batch):
          j = i+batch
          b = data.iloc[i:j]

          batch_recs = [tuple(row) for row in b.itertuples(index=False, name = None)]

          insert = f"""
          INSERT INTO {table}
          (PRODUCTID, PRODUCTNDC, PRODUCTTYPENAME, NONPROPRIETARYNAME,
          DOSAGEFORMNAME, STARTMARKETINGDATE, ENDMARKETINGDATE,
          MARKETINGCATEGORYNAME, LABELERNAME, SUBSTANCENAME,
          ACTIVE_NUMERATOR_STRENGTH, ACTIVE_INGRED_UNIT,
          DEASCHEDULE)
          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
          """
          cursor.executemany(insert, batch_recs)
          connection.commit()
          print(f"{j} records inserted")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection Closed")






Table Dropped
Creating New Table
Table Created
Inserting Data into df
Data Loaded into df
Starting insertion
500 records inserted
1000 records inserted
1500 records inserted
2000 records inserted
2500 records inserted
3000 records inserted
3500 records inserted
4000 records inserted
4500 records inserted
5000 records inserted
5500 records inserted
6000 records inserted
6500 records inserted
7000 records inserted
7500 records inserted
8000 records inserted
8500 records inserted
9000 records inserted
9500 records inserted
10000 records inserted
10500 records inserted
11000 records inserted
11500 records inserted
12000 records inserted
12500 records inserted
13000 records inserted
13500 records inserted
14000 records inserted
14500 records inserted
15000 records inserted
15500 records inserted
16000 records inserted
16500 records inserted
17000 records inserted
17500 records inserted
18000 records inserted
18500 records inserted
19000 records inserted
19500 records inserted
Connection Clo

In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.3 MB/s eta 0:00:00


In [2]:
from pymongo import MongoClient

hostname = "eshm34.h.filess.io"
database = "Healthdbnosql_mainwecard"
port = "61004"
username = "Healthdbnosql_mainwecard"
password = "0a5fb90224ef6634f3ada1741f7871b98bdb12e9"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

client = MongoClient(uri)

mydatabase = client[database]


In [3]:
import pandas as pd

try:
    condition_keyword_df = pd.read_csv('/content/condition_keyword_map.csv', encoding='utf-8')
except UnicodeDecodeError:
    condition_keyword_df = pd.read_csv('/content/condition_keyword_map.csv', encoding='latin-1')


data_to_insert = condition_keyword_df.to_dict('records')

mycollection = mydatabase['condition_keyword_map']

insert_res = mycollection.insert_many(data_to_insert)

print(f"Inserted {len(insert_res.inserted_ids)} documents into the 'condition_keyword_map' collection.")

Inserted 13 documents into the 'condition_keyword_map' collection.
